In [ ]:
import os
import pandas as pd
import numpy as np
from llama_cloud_services import LlamaExtract
from dotenv import load_dotenv
from typing import Optional
from pydantic import ValidationError

from pydantic import BaseModel, Field
from typing import Optional, Type

from BS_Schema_251017 import make_StatementOfFinancialPosition_model

In [ ]:
## Import Class
SFP = make_StatementOfFinancialPosition_model(2024)
# 4.2) Inspect its name and fields
print(SFP.__name__)
print(list(SFP.__fields__.keys()))

StatementOfFinancialPosition_2024
['cash_and_short_term_investments_unrestricted', 'cash_and_short_term_investments_restricted', 'accounts_receivable', 'pledges_receivable', 'government_grants_and_other_receivables', 'loans_receivable', 'all_receivables', 'accumulated_depreciation_bs', 'accumulated_amortization_bs', 'accumulated_depreciation_notes', 'accumulated_amortization_notes', 'rou_assets_finance_lease_bs', 'rou_assets_finance_lease_notes', 'net_fixed_assets_raw', 'rou_assets_operating_lease_bs', 'rou_assets_operating_lease_notes', 'long_term_investments', 'cash_surrender_value_life_insurance', 'total_assets', 'short_term_debt', 'accounts_payable', 'deferred_revenue_raw', 'student_tuition_and_deposits', 'student_credit_balances_and_deposits', 'asset_retirement_obligations', 'all_deferred_revenue', 'finance_lease_liability_bs', 'finance_lease_liability_notes', 'long_term_debt_labeled', 'other_long_term_debt_obligations', 'backup_total_long_term_debt', 'operating_lease_liability_bs

/var/folders/yc/4xwzhl2x2yq_cvnvnqf6hv580000gn/T/ipykernel_18025/528350060.py:5: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(list(SFP.__fields__.keys()))


In [ ]:
# PDF_ROOT = "private_universities/university_pdfs/"
# PDF_ROOT = "university_pdfs_hy"
PDF_ROOT = "university_pdfs_hy_3"
OUTPUT_ROOT = "output_Balance_Sheet_DS"
os.makedirs(OUTPUT_ROOT, exist_ok=True)
AGENT_ID = "bcb15a18-67ac-4772-9965-2654ecaff88c"
load_dotenv()
Year = 2024
extractor = LlamaExtract(api_key='llx-jlyOr0ZQwOzn0BPYgzggmo6mFShTLTsxdrXGviOaBf6IDnHG',project_id = '8c10e62e-3810-4193-915d-d2d11105826d')# LlamaExtract(project_id = '8c10e62e-3810-4193-915d-d2d11105826d')

# #uncomment the below line if you are creating the agent for the first time
# agent = extractor.create_agent(name = "balance-sheet-parser-v1", data_schema=SFP)

agent = extractor.get_agent(id = AGENT_ID)

# uncomment the following lines if you updated the schema
agent.data_schema = SFP
agent.save()
# agent = extractor.get_agent(id = AGENT_ID)

Extracting files: 100%|██████████| 1/1 [01:31<00:00, 91.37s/it]


In [4]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [6]:
# one row per file
results = {}

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    # 1) Gather all PDFs in the directory
    pdf_files = [
        f for f in sorted(os.listdir(school_dir))
        if f.lower().endswith(".pdf")
    ]

    # 2) Decide which PDFs to read:
    if len(pdf_files) == 1:
        to_read = pdf_files
    else:
        financial_only = [f for f in pdf_files if "financial" in f.lower()]
        to_read = financial_only if financial_only else pdf_files

    for fname in to_read:
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            results[(school, fname)] = data
        except Exception as err:
            print(f"Skipped {fname}: {err}")


Extracting data from ALFRED_UNIVERSITY/2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__364_KB_.pdf
Extracting data from AUGSBURG_UNIVERSITY/FY24_Augsburg_University_Financial_Statements_for_the_year_ended_05_31_2024__704_KB_.pdf
Extracting data from BRANDEIS_UNIVERSITY/2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__245_KB_.pdf
Extracting data from CHAPMAN_UNIVERSITY/Financial_Operating_Filing_for_the_year_ended_05_31_2024__361_KB_.pdf
Extracting data from COLLEGE_OF_ST_BENEDICT/Audited_Financial_Statements_for_the_year_ended_06_30_2024__416_KB_.pdf
Extracting data from CONNECTICUT_COLLEGE/Connecticut_College_for_the_year_ended_06_30_2024__319_KB_.pdf
Extracting data from CREIGHTON_UNIVERSITY/Annual_Audit_and_Operating_Data_for_the_year_ended_06_30_2024_Document1__468_KB_.pdf
Extracting data from ELIZABETHTOWN_COLLEGE/2024_Audited_Financial_Statements_-_Elizabethtown_College_for_the_year_ended_06_30_2024__889_KB_.pdf
Extracting data from FISHER_COLLEGE

In [ ]:
from typing import List

def add_plug_accounts(df: pd.DataFrame) -> pd.DataFrame:

    pairs_to_sum = {
        "rou_assets_finance_lease": ["rou_assets_finance_lease_bs", "rou_assets_finance_lease_notes"],
        "rou_assets_operating_lease": ["rou_assets_operating_lease_bs", "rou_assets_operating_lease_notes"],
        "finance_lease_liability": ["finance_lease_liability_bs", "finance_lease_liability_notes"],
        "operating_lease_liability": ["operating_lease_liability_bs", "operating_lease_liability_notes"],
    }
    
    for new_col, cols_to_sum in pairs_to_sum.items():
        if len(cols_to_sum) == 1:
            # Only one column → just copy it over
            df[new_col] = df[cols_to_sum[0]]

        else:
            print('placemaker 2: ', new_col)
            col1, col2 = cols_to_sum
            df[new_col] = df.apply(
                lambda row: (
                    row[col1] if pd.isna(row[col2]) else
                    row[col2] if pd.isna(row[col1]) else
                    row[col1] if row[col1] == row[col2] else
                    # row[col1] + row[col2]
                    row[col2]
                ),
                axis=1
            )

    # post-process net receivables
    net_receivables_components = [
        "accounts_receivable",
        "pledges_receivable",
        "government_grants_and_other_receivables",
        "loans_receivable"
    ]
    df["receivables_leftover_calculated"] = df["all_receivables"] - df[net_receivables_components].fillna(0).sum(axis=1)
    df["net_receivables"] = df[net_receivables_components].fillna(0).sum(axis=1) + df["receivables_leftover_calculated"]
    
    df["pension_and_opeb_liability"] = df.get("pension_liability", 0).fillna(0) + df.get("opeb_liability", 0).fillna(0)

    # post-process accumulated depreciation
    df["accumulated_depreciation"] = (
        # Depreciation: use one if bs == notes to avoid double counting
    np.where(
        df["accumulated_depreciation_bs"].fillna(0) 
        == df["accumulated_depreciation_notes"].fillna(0),
        df["accumulated_depreciation_bs"].fillna(0),
        df["accumulated_depreciation_bs"].fillna(0)
        + df["accumulated_depreciation_notes"].fillna(0),
    )
    +
        # Amortization: same rule
    np.where(
        df["accumulated_amortization_bs"].fillna(0)
        == df["accumulated_amortization_notes"].fillna(0),
        df["accumulated_amortization_bs"].fillna(0),
        df["accumulated_amortization_bs"].fillna(0)
        + df["accumulated_amortization_notes"].fillna(0),
    )
)
    # post-process long term investments
    df["long_term_investments_unrestricted_and_restricted"] = (
        df.get("long_term_investments", 0).fillna(0)
        + df.get("cash_surrender_value_life_insurance", 0).fillna(0)
    )

    # post-process deferred revenue
    a = df.get("deferred_revenue_raw", 0).fillna(0)
    b = df.get("student_tuition_and_deposits", 0).fillna(0)
    c = df.get("student_credit_balances_and_deposits", 0).fillna(0)
    # Stack components row-wise and drop duplicates
    df["deferred_revenue"] = (
        pd.DataFrame({"a": a, "b": b, "c": c})
        .apply(lambda x: sum(set(x[x != 0])), axis=1)
    )
    df["deferred_revenue"] -= df.get("asset_retirement_obligations", 0).fillna(0) * df.get("asset_retirement_obligations", 0).fillna(0)
    df["all_deferred_revenue"] -= df.get("asset_retirement_obligations", 0).fillna(0) * df.get("asset_retirement_obligations", 0).fillna(0)

    df["long_term_debt"] = (
        df.get("long_term_debt_labeled", 0).fillna(0)
        + df.get("other_long_term_debt_obligations", 0).fillna(0)
        + df.get("finance_lease_liability", 0).fillna(0)
    )

    df["long_term_debt"] = df["long_term_debt"].where(
    df["long_term_debt"] != 0,
    df.get("backup_total_long_term_debt", 0).fillna(0)
)  

    df["net_fixed_assets"] = df["net_fixed_assets_raw"].fillna(0) + df["rou_assets_finance_lease"].fillna(0)

    asset_components = [
        "cash_and_short_term_investments_unrestricted_and_restricted",
        "net_receivables",
        "net_fixed_assets",
        "long_term_investments_unrestricted_and_restricted",
        "rou_assets_operating_lease"
    ]
    liability_components = [
        "short_term_debt",
        "current_portion_finance_lease",
        "current_portion_long_term_debt",
        "current_portion_operating_lease",
        "accounts_payable",
        "all_deferred_revenue",
        "long_term_debt",
        "finance_lease_liability",
        "operating_lease_liability",
        "swap_obligation_fmv",
        "pension_and_opeb_liability"
    ]
    
    assets_cols = [c for c in asset_components if c in df.columns]
    liabs_cols  = [c for c in liability_components if c in df.columns]
    
    df["sum_asset_components"]     = df[assets_cols].sum(axis=1, skipna=True)
    df["sum_liability_components"] = df[liabs_cols].sum(axis=1, skipna=True)
    
    df["other_assets_plug"]     = df["total_assets"] - df["sum_asset_components"]
    df["other_liabilities_plug"] = df["total_liabilities"] - df["sum_liability_components"]
    df["expendable_net_assets_with_donor_restrictions"] = df["net_assets_with_donor_restrictions"] - df["perpetual_net_assets_with_donor_restrictions"]

    
    return df


In [8]:
# # cont. one row per file

# OUTPUT_FILE = "unprocessed_all_private_universities_balance_sheet.xlsx"

# # save output
# add_plug_accounts(df_all).to_excel(OUTPUT_FILE)

In [9]:
# Build DataFrame: one row per (school, filename)
df_all = pd.DataFrame.from_dict(results, orient="index")

# optional: give the index clearer names
df_all.index = pd.MultiIndex.from_tuples(df_all.index, names=["school", "file"])

# add plug accounts and make adjustment for expendable assets
df_allv1 = add_plug_accounts(df_all)

placemaker 2:  rou_assets_finance_lease
placemaker 2:  rou_assets_operating_lease
placemaker 2:  finance_lease_liability
placemaker 2:  operating_lease_liability


In [ ]:
desired_order = [
    "year", 
    "cash_and_short_term_investments_unrestricted",
    "cash_and_short_term_investments_restricted",
    "net_receivables",
    "accumulated_depreciation",
    "rou_assets_finance_lease",
    "net_fixed_assets",
    "rou_assets_operating_lease", 
    "long_term_investments_unrestricted_and_restricted",
    "other_assets_plug",
    "total_assets", 
    "short_term_debt", "accounts_payable", 
    "deferred_revenue",
    "all_deferred_revenue", 
    "finance_lease_liability",
    "long_term_debt",
    "operating_lease_liability", 
    "swap_obligation_fmv", 
    "pension_and_opeb_liability", 
    "other_liabilities_plug",
    "total_liabilities",
    "net_assets_without_donor_restrictions", 
    "expendable_net_assets_with_donor_restrictions",
    "perpetual_net_assets_with_donor_restrictions", 
    "net_assets_with_donor_restrictions",
    "noncontrolling_interest",
    "total_net_assets", 
    "total_liabilities_and_net_assets",

    
    "cash_and_short_term_investments_unrestricted", 
    "cash_and_short_term_investments_restricted",
    
    "accounts_receivable", "pledges_receivable", "government_grants_and_other_receivables", 
    "loans_receivable","receivables_leftover_calculated",
    "net_receivables",
    
    "accumulated_depreciation_bs", "accumulated_amortization_bs",
    "accumulated_depreciation_notes", "accumulated_amortization_notes", "accumulated_depreciation", 
    
    "rou_assets_finance_lease_bs", "rou_assets_finance_lease_notes", "rou_assets_finance_lease", 
    
    "net_fixed_assets_raw", "net_fixed_assets",

    "rou_assets_operating_lease_bs", "rou_assets_operating_lease_notes",
    "rou_assets_operating_lease", 

    "long_term_investments", "cash_surrender_value_life_insurance",
    "long_term_investments_unrestricted_and_restricted",

    "other_assets_plug", 
    "total_assets", 

    "current_portion_finance_lease",
    "current_portion_long_term_debt", "current_portion_operating_lease", 

    "short_term_debt", "accounts_payable", 

    # "deferred_revenue_raw", "student_tuition_and_deposits", "student_credit_balances_and_deposits",
    # "deferred_revenue", 
    "all_deferred_revenue", "asset_retirement_obligations",

    "finance_lease_liability_bs", "finance_lease_liability_notes", "finance_lease_liability", 

    "long_term_debt_labeled", "other_long_term_debt_obligations",
    # "bonds_payable",
    "long_term_debt",

    "operating_lease_liability_bs", "operating_lease_liability_notes",
    "operating_lease_liability", 

    "swap_obligation_fmv", 

    "pension_liability", "opeb_liability", "pension_and_opeb_liability", 

    "other_liabilities_plug", "total_liabilities",


    "net_assets_without_donor_restrictions", 
    "expendable_net_assets_with_donor_restrictions",
    "perpetual_net_assets_with_donor_restrictions", 
    "net_assets_with_donor_restrictions",
    "noncontrolling_interest",

    "total_net_assets", 
    "total_liabilities_and_net_assets",
    
    "contribution_type",
    "backup_total_long_term_debt",
]

remaining_columns = [col for col in df_allv1.columns if col not in desired_order]
full_column_order = desired_order + remaining_columns

df_reordered = df_allv1[full_column_order]

# df_reordered = df_allv1.reindex(columns=desired_order)

In [11]:
# OUTPUT_FILE = "all_private_universities_balance_sheet.xlsx"
OUTPUT_FILE = "more_universities_second_half_bs_1013_3.xlsx"

# save output
df_reordered.to_excel(OUTPUT_FILE)